In [10]:
# Imports
import requests
import time
import re
import pandas as pd
import pickle as pkl
import os

In [2]:
# Only in colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Search Instruments
url = 'https://api.tdameritrade.com/v1/instruments'

df = pd.read_csv('/content/drive/MyDrive/stockScreener/company_list.csv')
symbols = df['Symbol'].values.tolist()

In [4]:
from secrets import Consumer_Key
# Consumer_Key

In [17]:
# Merging and extracting
start = 0
end = 500
files = []
while start < len(symbols):
    tickers = symbols[start: end]
    payload = {'apikey': Consumer_Key,
               'symbol': tickers,
               'projection': 'fundamental'}

    results = requests.get(url, params=payload)
    data = results.json()
    f_name = time.asctime() + '.pkl'
    f_name = re.sub('[ :]', '_', f_name)
    files.append(f_name)
    with open(f_name, 'wb') as file:
        pkl.dump(data, file)

    start = end
    end += 500
    time.sleep(1)

In [18]:
data = []

for file in files:
    with open(file, 'rb') as f:
        info = pkl.load(f)
    tickers = list(info)
    points = ['symbol', 'netProfitMarginMRQ', 'peRatio', 'pegRatio', 'high52']
    for ticker in tickers:
        tick = []
        for point in points:
            tick.append(info[ticker]['fundamental'][point])
        data.append(tick)
    os.remove(file)

In [19]:
points = ['symbol', 'Margin', 'PE', 'PEG', 'high52']

df_results = pd.DataFrame(data, columns=points)

df_peg = df_results[
    (df_results['PEG'] < 1) &
    (df_results['PEG'] > 0) &
    (df_results['Margin'] > 20) &
    (df_results['PE'] > 10)]

df_peg.sort_values(['PEG'])

pd.set_option('display.max_rows', 2000)

In [20]:
"""
# Function to see data in tranches instead of one set
def view(size):
    start = 0
    stop = size
    while stop < len(df_peg):
        print(df_peg[start: stop])
        start = stop
        stop += size
    print(df_peg[start: stop])

view(35)
"""

'\n# Function to see data in tranches instead of one set\ndef view(size):\n    start = 0\n    stop = size\n    while stop < len(df_peg):\n        print(df_peg[start: stop])\n        start = stop\n        stop += size\n    print(df_peg[start: stop])\n\nview(35)\n'

In [21]:
df_symbols = df_peg['symbol'].tolist()

new = df['Symbol'].isin(df_symbols)

companies = df[new]
companies.reset_index()

print(companies)

      Symbol  ...                                           Industry
104     ACTG  ...                              Professional Services
151     ATVI  ...                                      Entertainment
192     ADBE  ...                                           Software
210      AMD  ...           Semiconductors & Semiconductor Equipment
211     AOXY  ...               Real Estate Management & Development
257      AMG  ...                                    Capital Markets
327     AIXN  ...                                       Distributors
372      ARE  ...       Equity Real Estate Investment Trusts (REITs)
420       AB  ...                                    Capital Markets
423      NCV  ...                                                NaN
424      NCZ  ...                                                NaN
456      APT  ...                                  Building Products
497       MO  ...                                            Tobacco
599     AMNB  ...                 